In [35]:
import tensorflow as tf
import numpy as np
from utilities import my_callbacks
from utilities import data_helper
import optparse
import sys
import math

In [36]:
vocab = data_helper.load_all(filelist="final_data/wsj.all")
print(vocab)

Using features: None
{'-': 2449404, 'X': 162058, 'S': 52415, 'O': 30440}
Total vocabulary size in the whole dataset: 4
['-', 'O', 'S', 'X', '0']


In [37]:
print("loading entity-gird for pos and neg documents...")

X_train_1, X_train_0, E = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.dev", 
        perm_num = 20, maxlen=2000, window_size=6, vocab_list=vocab, emb_size=100)

X_dev_1, X_dev_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.dev", 
        perm_num = 20, maxlen=2000, window_size=6, vocab_list=vocab, emb_size=100)

X_test_1, X_test_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.dev", 
        perm_num = 20, maxlen=2000, window_size=6, vocab_list=vocab, emb_size=100)


loading entity-gird for pos and neg documents...


In [38]:
num_train = len(X_train_1)
num_dev   = len(X_dev_1)
num_test  = len(X_test_1)
#assign Y value
y_train_1 = [1] * num_train 
y_dev_1 = [1] * num_dev 
y_test_1 = [1] * num_test 

print('.....................................')
print("Num of traing pairs: " + str(num_train))
print("Num of dev pairs: " + str(num_dev))
print("Num of test pairs: " + str(num_test))
#print("Num of permutation in train: " + str(opts.p_num)) 
#print("The maximum in length for CNN: " + str(opts.maxlen))
print('.....................................')

.....................................
Num of traing pairs: 2678
Num of dev pairs: 2678
Num of test pairs: 2678
.....................................


In [39]:
X_train_1.shape

(2678, 2000)

In [40]:
def forward_propagation(X_positive, X_negative, vocab):
    """
    Implements forward propagation of Neural coherence model
    
    Args:
    X_positive - A Placeholder for positive document
    X_negative - A Placeholder for negative document
    vocab - Vocabulary list of entire entity grid list
    
    Returns: 
    out_positive: Coherence Score for positive document
    out_negative: Coherence Score for negative document
    
    """
    
    ## Placeholders
    #X_positive = tf.placeholder(tf.int32, shape = [None, 2000]) #opts.maxlen=2000
    #X_negative = tf.placeholder(tf.int32, shape = [None, 2000]) #opts.maxlen=2000
    

    ## First Layer of NN: Transform each grammatical role in the grid into distributed representation - a real valued vector
    
    
    #Shared embedding matrix
    W_embedding = tf.get_variable("W_embedding", [len(vocab), 100], initializer = tf.contrib.layers.xavier_initializer()) #embedding matrix 
        
    
    #Look up layer
    
    #for positive document
    embedding_positive = tf.nn.embedding_lookup(W_embedding, X_positive)
    
    #for negative document
    embedding_negative = tf.nn.embedding_lookup(W_embedding, X_negative)


    ## Second Layer of NN: Convolution Layer
    
    
    #shared filter and bias
    w_size = 6       #filter_size
    emb_size = 100   #embedding_size 
    nb_filter = 150  #num_filters 

    filter_shape = [w_size, emb_size, nb_filter]

    W_conv_layer_1 = tf.get_variable("W_conv_layer_1", shape = filter_shape, initializer = tf.contrib.layers.xavier_initializer(seed = 0)) #filter for covolution layer 1
    b_conv_layer_1 =  tf.get_variable("b_conv_layer_1", shape=[nb_filter], initializer = tf.constant_initializer(0.0))  #bias for convolution layer 1

    
    #1D Convolution for positive document
    conv_layer_1_positive = tf.nn.conv1d(embedding_positive, W_conv_layer_1, stride=1, padding="VALID")  #embedding and W_conv_layer_1 both are 3D matrix
    conv_layer_1_with_bias_positive = tf.nn.bias_add(conv_layer_1_positive, b_conv_layer_1)    
    h_conv_layer_1_positive = tf.nn.relu(conv_layer_1_with_bias_positive, name="relu_conv_layer_1_positive") # Apply nonlinearity
    
    
    #1D Convolution for negative document
    conv_layer_1_negative = tf.nn.conv1d(embedding_negative, W_conv_layer_1, stride=1, padding="VALID")  #embedding and W_conv_layer_1 both are 3D matrix
    conv_layer_1_with_bias_negative = tf.nn.bias_add(conv_layer_1_negative, b_conv_layer_1)    
    h_conv_layer_1_negative = tf.nn.relu(conv_layer_1_with_bias_negative, name="relu_conv_layer_1_negative") # Apply nonlinearity

    

    ## Third Layer of NN: Pooling Layer
    
    
    #1D Pooling for positive document
    m_layer_1_positive = tf.nn.pool(h_conv_layer_1_positive, window_shape = [6], strides = [6], pooling_type = 'MAX', padding="VALID")

    #1D Pooling for negative document
    m_layer_1_negative = tf.nn.pool(h_conv_layer_1_negative, window_shape = [6], strides = [6], pooling_type = 'MAX', padding="VALID")



    ## Fourth Layer of NN: Fully Connected Layer

    
    #Flatten
    
    #for positive document
    flatten_positive = tf.contrib.layers.flatten(m_layer_1_positive)
    
    #for negative document
    flatten_negative = tf.contrib.layers.flatten(m_layer_1_negative)
    
    

    #Dropout
    
    #for positive document
    drop_out_positive = tf.nn.dropout(flatten_positive, keep_prob=0.5)
    
    #for negative document
    drop_out_negative = tf.nn.dropout(flatten_negative, keep_prob=0.5)

    
    
    # Coherence Scoring
    
    #for positive document
    out_positive = tf.contrib.layers.fully_connected(drop_out_positive, num_outputs = 1, activation_fn=None)
    #out_positive = tf.sigmoid(out_positive)
    
    #for negative document
    out_negative = tf.contrib.layers.fully_connected(drop_out_negative, num_outputs = 1, activation_fn=None)
    #out_negative = tf.sigmoid(out_negative)
    
    return out_positive, out_negative
    

In [41]:
def ranking_loss(pos, neg):
    loss = tf.maximum(1.0 + neg - pos, 0.0) 
    #print(loss)
    return tf.reduce_mean(loss)

In [42]:
def mini_batches(X, Y, m, mini_batch_size = 32):
        
    mini_batches = []
    
    num_complete_minibatches = math.floor(m/mini_batch_size) 
    
    for k in range(0, num_complete_minibatches):
        mini_batch_X = X[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :]
        mini_batch_Y = Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = X[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch_Y = Y[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [43]:
tf.reset_default_graph()

In [44]:
## Create Placeholders
X_positive = tf.placeholder(tf.int32, shape = [None, 2000]) #Placeholder for positive document
X_negative = tf.placeholder(tf.int32, shape = [None, 2000]) #Placeholder for negative document

# Forward propagation
score_positive, score_negative = forward_propagation(X_positive, X_negative, vocab)
    
# Cost function:
cost = ranking_loss(score_positive, score_negative)

# Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)


In [46]:
init = tf.global_variables_initializer()

num_epochs = 10
minibatch_size = 32
m = num_train

with tf.Session() as sess:

    sess.run(init)

    for epoch in range(num_epochs):

        minibatch_cost = 0.
        num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
        minibatches = mini_batches(X_train_1, X_train_0, m, minibatch_size)

        for (i, minibatch) in enumerate(minibatches):

            (minibatch_X_positive, minibatch_X_negative) = minibatch
            
            _ , temp_cost, pos, neg = sess.run([optimizer, cost, score_positive, score_negative], 
                        feed_dict={X_positive:minibatch_X_positive, 
                                   X_negative:minibatch_X_negative})
            print("Epoch:", epoch, "Minibatch:", i) 
            print("Positive score:", pos) 
            print("Negative score:", neg)
            print("ranking loss:", temp_cost)
            
            #print("*************** End of a minibatch **********************************")
            
            #print(temp_cost)
            minibatch_cost += temp_cost / num_minibatches
        
        print(minibatch_cost)
        print("******************* End of an epoch ******************************")

Epoch: 0 Minibatch: 0
Positive score: [[  2.01787472e-01]
 [  3.36630821e-01]
 [ -1.08408302e-01]
 [ -9.90448222e-02]
 [  1.18104666e-01]
 [  1.63117155e-01]
 [ -5.61306775e-02]
 [ -1.26685724e-01]
 [ -1.93822429e-01]
 [  3.52824591e-02]
 [ -5.38243055e-02]
 [ -1.96992442e-01]
 [ -8.31028223e-02]
 [  6.57005608e-03]
 [ -2.14558005e-01]
 [  2.33509973e-01]
 [  1.61114991e-01]
 [ -2.50006467e-02]
 [  3.54880035e-01]
 [ -3.23839486e-04]
 [ -1.90162048e-01]
 [ -2.58072197e-01]
 [ -4.44089361e-02]
 [ -9.33392644e-02]
 [ -2.08169520e-01]
 [ -1.92278460e-01]
 [ -3.16656291e-01]
 [ -2.69411087e-01]
 [ -4.03842032e-01]
 [ -2.81395093e-02]
 [ -1.02413759e-01]
 [ -1.99048921e-01]]
Negative score: [[ 0.40600222]
 [ 0.04682779]
 [ 0.29847685]
 [ 0.31638655]
 [ 0.07263301]
 [ 0.52392948]
 [ 0.0560288 ]
 [ 0.13496928]
 [-0.12247022]
 [ 0.28872758]
 [ 0.01746091]
 [ 0.03353466]
 [ 0.28971329]
 [-0.0560744 ]
 [ 0.36116198]
 [ 0.0127331 ]
 [ 0.22164276]
 [ 0.05558525]
 [-0.00367223]
 [-0.04277167]
 [ 0.

Epoch: 0 Minibatch: 8
Positive score: [[ 21.2482872 ]
 [ 21.17472649]
 [ 20.87641144]
 [ 21.07275772]
 [ 21.29427719]
 [ 20.85260201]
 [ 21.23572922]
 [ 21.47337532]
 [ 19.48334503]
 [ 19.66023254]
 [ 19.29915428]
 [ 20.00741196]
 [ 18.70795059]
 [ 19.65415955]
 [ 19.24312973]
 [ 19.36201096]
 [ 19.49515724]
 [ 20.04730606]
 [ 19.50087738]
 [ 19.61100578]
 [ 18.54636383]
 [ 19.66031265]
 [ 19.59446335]
 [ 19.81318283]
 [ 19.40830421]
 [ 19.39560318]
 [ 20.00474739]
 [ 19.61208344]
 [ 18.37159729]
 [ 18.73344421]
 [ 18.39740753]
 [ 19.07433128]]
Negative score: [[-22.1716156 ]
 [-22.15218735]
 [-22.08231926]
 [-22.10188293]
 [-22.56995392]
 [-22.07457352]
 [-22.3127861 ]
 [-22.05901527]
 [-20.55039597]
 [-20.60977936]
 [-20.23290825]
 [-20.14957047]
 [-20.65376472]
 [-20.57135773]
 [-21.06211662]
 [-20.4626255 ]
 [-20.64121628]
 [-20.81806564]
 [-20.76536179]
 [-20.52804565]
 [-20.42004395]
 [-20.84440422]
 [-20.95517731]
 [-20.27996635]
 [-20.85654831]
 [-20.5899086 ]
 [-20.57728958]
 

Epoch: 0 Minibatch: 16
Positive score: [[ 32.53502655]
 [ 32.31330872]
 [ 32.29660034]
 [ 32.8527832 ]
 [ 32.46154785]
 [ 33.61610031]
 [ 32.78372955]
 [ 33.35146332]
 [ 32.90839386]
 [ 35.50583267]
 [ 35.24649048]
 [ 35.38195038]
 [ 35.54868698]
 [ 35.47303009]
 [ 35.82258224]
 [ 36.27255249]
 [ 35.20083618]
 [ 35.45510864]
 [ 35.21833801]
 [ 35.46055222]
 [ 35.51683044]
 [ 35.53553391]
 [ 35.9894371 ]
 [ 35.64810181]
 [ 35.16561508]
 [ 35.09814453]
 [ 35.23488617]
 [ 36.41668701]
 [ 36.06747818]
 [ 29.31634712]
 [ 28.76281357]
 [ 29.50979233]]
Negative score: [[-34.64221191]
 [-34.12283707]
 [-34.73511505]
 [-34.99289703]
 [-35.12576294]
 [-34.26766205]
 [-33.95188904]
 [-34.46466064]
 [-34.3819046 ]
 [-38.09536362]
 [-38.1939888 ]
 [-38.08993149]
 [-37.918396  ]
 [-37.35918045]
 [-37.68260193]
 [-37.13636398]
 [-36.70962524]
 [-37.02566528]
 [-37.65364838]
 [-37.62754822]
 [-37.42563248]
 [-37.17655182]
 [-37.22439194]
 [-37.65611267]
 [-37.50699615]
 [-37.20970917]
 [-37.28941345]


Epoch: 0 Minibatch: 24
Positive score: [[ 36.05535507]
 [ 36.36399078]
 [ 36.82925415]
 [ 36.22298431]
 [ 35.70466232]
 [ 35.27931595]
 [ 36.06320953]
 [ 36.24424744]
 [ 35.86906815]
 [ 35.29374313]
 [ 35.15724945]
 [ 35.55754852]
 [ 35.77045822]
 [ 35.67990112]
 [ 36.21684265]
 [ 35.10328674]
 [ 35.03638077]
 [ 35.51133728]
 [ 35.72376251]
 [ 35.41725159]
 [ 35.77072906]
 [ 34.95707703]
 [ 35.35220337]
 [ 35.463974  ]
 [ 34.46281433]
 [ 35.12928772]
 [ 35.69373322]
 [ 35.72525787]
 [ 35.4936676 ]
 [ 34.94158936]
 [ 36.45565796]
 [ 35.89797974]]
Negative score: [[-36.95702744]
 [-39.01264191]
 [-37.99146271]
 [-38.25772095]
 [-37.93920898]
 [-37.74945831]
 [-38.23875809]
 [-37.97688293]
 [-37.87885284]
 [-38.02986526]
 [-36.61399078]
 [-37.91437912]
 [-36.79295349]
 [-37.57069778]
 [-37.86492157]
 [-36.67259216]
 [-37.10586548]
 [-36.90689087]
 [-37.43437195]
 [-37.11967468]
 [-37.45185089]
 [-37.88132095]
 [-37.23246765]
 [-36.88022614]
 [-37.92993927]
 [-37.35243988]
 [-37.1068573 ]


Epoch: 0 Minibatch: 32
Positive score: [[ 39.09186935]
 [ 38.80164337]
 [ 38.43585587]
 [ 39.10123825]
 [ 38.75311279]
 [ 37.62998581]
 [ 38.60951614]
 [ 38.4847908 ]
 [ 38.64936447]
 [ 39.22309494]
 [ 37.73428726]
 [ 39.93296814]
 [ 39.50734711]
 [ 40.05340195]
 [ 39.25931549]
 [ 39.57963943]
 [ 39.20930862]
 [ 40.11201859]
 [ 39.34064865]
 [ 39.7419014 ]
 [ 40.09891129]
 [ 38.72274399]
 [ 39.16661453]
 [ 38.54227829]
 [ 39.76942825]
 [ 39.62637329]
 [ 39.2725563 ]
 [ 39.18096542]
 [ 39.72909927]
 [ 39.14757919]
 [ 39.11416626]
 [ 38.01978683]]
Negative score: [[-41.04621506]
 [-40.53226089]
 [-40.80285263]
 [-40.50427628]
 [-41.13975143]
 [-40.91527557]
 [-41.16822052]
 [-40.53430939]
 [-40.31758118]
 [-40.21656036]
 [-40.67125702]
 [-42.05228806]
 [-42.44520187]
 [-42.08079147]
 [-42.20783615]
 [-42.33915329]
 [-42.27262115]
 [-41.99318695]
 [-41.05060196]
 [-42.65589523]
 [-41.06975555]
 [-41.44371796]
 [-41.53949356]
 [-41.25054932]
 [-42.26134109]
 [-41.97268677]
 [-41.61787796]


...

Epoch: 7 Minibatch: 68
Positive score: [[ 41.41594315]
 [ 41.5631752 ]
 [ 41.21014404]
 [ 39.73820114]
 [ 39.9506073 ]
 [ 39.92086792]
 [ 39.81459808]
 [ 39.59184647]
 [ 39.65170288]
 [ 40.0416069 ]
 [ 40.01062775]
 [ 40.2598114 ]
 [ 39.64329529]
 [ 38.76856995]
 [ 39.84266663]
 [ 40.48980713]
 [ 39.84675598]
 [ 39.68423462]
 [ 39.37173843]
 [ 39.61618423]
 [ 40.6269722 ]
 [ 39.35852814]
 [ 48.06895447]
 [ 47.16280365]
 [ 47.86047363]
 [ 47.90750885]
 [ 47.81977844]
 [ 47.73484802]
 [ 47.65775681]
 [ 47.91977692]
 [ 48.15753937]
 [ 48.78812408]]
Negative score: [[-44.01914978]
 [-44.29232788]
 [-44.06848526]
 [-42.0993042 ]
 [-41.9032402 ]
 [-41.66713333]
 [-41.74629974]
 [-42.52952576]
 [-42.51178741]
 [-41.59398651]
 [-42.73677826]
 [-41.75001907]
 [-42.39446259]
 [-42.53968811]
 [-41.50826645]
 [-42.23752594]
 [-42.63687134]
 [-41.70137405]
 [-42.14382172]
 [-41.98984146]
 [-42.36048889]
 [-42.29187012]
 [-50.49682617]
 [-50.98799896]
 [-49.83062744]
 [-51.76862335]
 [-51.09334564]


Epoch: 7 Minibatch: 76
Positive score: [[ 41.05139923]
 [ 39.79654694]
 [ 39.42742157]
 [ 38.78122711]
 [ 39.44689941]
 [ 39.72360229]
 [ 40.3626442 ]
 [ 40.44367599]
 [ 40.26548767]
 [ 40.82368469]
 [ 41.05222702]
 [ 40.71840286]
 [ 40.21261978]
 [ 40.13916779]
 [ 40.84770203]
 [ 40.73918533]
 [ 41.59797668]
 [ 40.30230713]
 [ 40.92996216]
 [ 41.7811203 ]
 [ 40.5800705 ]
 [ 41.70724106]
 [ 41.03466034]
 [ 40.8941803 ]
 [ 40.6552124 ]
 [ 41.32339096]
 [ 41.64106369]
 [ 42.80266571]
 [ 42.36384583]
 [ 41.9336586 ]
 [ 42.4388504 ]
 [ 42.43209839]]
Negative score: [[-42.28804779]
 [-40.95681763]
 [-42.15005875]
 [-41.7360878 ]
 [-41.74957275]
 [-42.24697495]
 [-42.06681061]
 [-42.07429886]
 [-41.79210281]
 [-43.18130112]
 [-42.32943344]
 [-42.68120193]
 [-43.5181427 ]
 [-43.45411301]
 [-43.3132019 ]
 [-42.68326569]
 [-42.87229156]
 [-43.10287857]
 [-43.15405273]
 [-43.0404129 ]
 [-42.94716644]
 [-42.82990646]
 [-42.18270493]
 [-42.48899841]
 [-43.12089157]
 [-42.95441437]
 [-44.32301331]


Epoch: 8 Minibatch: 0
Positive score: [[ 48.07000732]
 [ 48.00791931]
 [ 47.89798355]
 [ 48.00052643]
 [ 47.73567581]
 [ 48.38998413]
 [ 48.22665024]
 [ 48.78556061]
 [ 47.67363739]
 [ 48.17329788]
 [ 48.22915649]
 [ 48.10194397]
 [ 48.66953659]
 [ 47.44707489]
 [ 48.21074295]
 [ 48.32671356]
 [ 48.50892258]
 [ 48.16138458]
 [ 48.79425812]
 [ 48.46441269]
 [ 48.3881073 ]
 [ 49.20373535]
 [ 49.89255524]
 [ 48.67912674]
 [ 49.56108856]
 [ 48.6725235 ]
 [ 49.17012405]
 [ 49.15772247]
 [ 49.31309891]
 [ 48.73191071]
 [ 49.41143036]
 [ 49.47174072]]
Negative score: [[-51.00246429]
 [-49.91913223]
 [-50.83571625]
 [-49.64411163]
 [-50.42269897]
 [-51.15505219]
 [-48.68292999]
 [-51.01379776]
 [-50.20800781]
 [-49.69744873]
 [-50.61053467]
 [-50.16479492]
 [-50.32289124]
 [-51.11677933]
 [-50.22065735]
 [-51.78461456]
 [-50.28658676]
 [-50.74274063]
 [-50.3147049 ]
 [-50.33140182]
 [-51.61762238]
 [-50.93121719]
 [-51.93732452]
 [-51.08627701]
 [-52.05301285]
 [-51.65654755]
 [-50.78419495]
 

Epoch: 8 Minibatch: 8
Positive score: [[ 43.58211899]
 [ 44.29350662]
 [ 45.28503036]
 [ 44.1971817 ]
 [ 45.3135643 ]
 [ 45.01303101]
 [ 44.15081787]
 [ 45.57124329]
 [ 42.52146912]
 [ 42.40932465]
 [ 42.78279877]
 [ 42.81880951]
 [ 43.29765701]
 [ 42.56479645]
 [ 42.03520203]
 [ 42.85240936]
 [ 42.17326736]
 [ 42.63332748]
 [ 42.05005646]
 [ 42.43212128]
 [ 43.05708313]
 [ 42.74108505]
 [ 41.82572174]
 [ 42.96077728]
 [ 42.54225159]
 [ 41.76977921]
 [ 43.1337738 ]
 [ 41.82095337]
 [ 40.49733734]
 [ 41.26335907]
 [ 40.60216904]
 [ 41.15917206]]
Negative score: [[-48.08798599]
 [-47.83764267]
 [-47.4193573 ]
 [-48.24329758]
 [-48.32911682]
 [-47.85007477]
 [-46.59881592]
 [-46.99034119]
 [-44.45760345]
 [-45.02304077]
 [-44.37259293]
 [-45.04651642]
 [-44.73847961]
 [-45.57611084]
 [-44.10843658]
 [-44.85900497]
 [-44.86077118]
 [-45.04219055]
 [-45.26839447]
 [-45.02943802]
 [-44.91376114]
 [-44.86862946]
 [-44.65352631]
 [-44.48804092]
 [-46.32769012]
 [-44.05401611]
 [-44.57351303]
 

KeyboardInterrupt: 